# Sentiment analysis
<img src="./screencast.gif"/>

In this sample, we will build a sentiment annotator for the [Movie Review](http://www.cs.cornell.edu/people/pabo/movie-review-data/) dataset from Cornell.

First we extract the tarball

In [1]:
!tar xfzv data.json.tgz

x data.json


In [2]:
import json
with open('data.json') as f:
    data = json.load(f)

There's a lot of data here, lets process a subset of it.

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

data = [{'text': _['text'], 'sentiment': sentiment_analyzer.polarity_scores(_['text'])['compound']} for _ in list(data.values())[:10]]

/usr/local/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


We will write a simple display formatter to make our output look nice

In [4]:
from IPython.display import display, HTML, Code
def display_record(record):
    sentiment = 1 if record['sentiment'] > 0 else -1
    if sentiment == 1:
        display(HTML('<span style="color:green;">{}</span>'.format(sentiment)))
    else:
        display(HTML('<span style="color:red;">{}</span>'.format(sentiment)))
    print(record['text'])

display_record(data[0])

in my review of " the spy who shagged me , " i postulated an unbreakable law of film physics : every time a sequel is as good as or better than the previous film in the series , it is followed by a third movie that is a bore . 
the cause is probably complacency ; a studio sighs with relief when part 2 lives up to expectations and figures part 3 is a sure thing . 
 " scream 3 " provides the latest proof of this rule . 
in los angeles production has begun on " stab 3 : return to woodsboro , " the most recent installment in the series of movies inspired by the murders surrounding sidney prescott ( neve campbell ) . 
however , life soon starts imitating art , and " stab " cast members turn up stabbed . 
smelling yet another book deal , gale weathers ( courteney cox arquette ) comes to the set to investigate and finds her ex-boyfriend dewey riley ( david arquette ) acting as a technical consultant and getting chummy with jennifer ( parker posey ) , the actress playing gale in " stab 3 . " e

## Assemble our annotator
Now we can assemble our checker using `ipyannotate`. For this task, we will show the user the model-evaluated sentiment, and let them override it with `+1`, `0` and `-1` buttons, which will modify the annotation tasks.

In [5]:
from ipyannotate.buttons import OkButton as Button, NextButton, BackButton
from ipyannotate.toolbar import Toolbar
from ipyannotate.tasks import Task, Tasks
from ipyannotate.canvas import OutputCanvas
from ipyannotate.annotation import Annotation


def handle_click(button):
    annotation.tasks.current.output['sentiment'] = button.value

tasks = Tasks(Task(_) for _ in data)

pos = Button(label='+', shortcut='1', value=1, color='green', icon='', callback=handle_click)
neu = Button(label='o', shortcut='2', value=0, color='gray', icon='', callback=handle_click)
neg = Button(label='-', shortcut='3', value=-1, color='red', icon='', callback=handle_click)

buttons = [pos, neu, neg, BackButton(shortcut='j'), NextButton(shortcut='k')]
toolbar = Toolbar(buttons)

canvas = OutputCanvas(display=display_record)

annotation = Annotation(toolbar, tasks, canvas=canvas)
annotation

Annotation(canvas=OutputCanvas(), progress=Progress(atoms=[<ipyannotate.progress.Atom object at 0x1074e2898>, …

# annotation.tasks

In [6]:
annotation.tasks

[Task(output={'text': 'in my review of " the spy who shagged me , " i postulated an unbreakab..., value=None),
 Task(output={'text': 'five years after his directorial debut based on stephen king\'s writin..., value=None),
 Task(output={'text': 'i went to blair witch project 2 : book of shadows with the highest of ..., value=None),
 Task(output={'text': 'the word to describe sharon stone is " wonder " . \nnot that she _is_ ..., value=None),
 Task(output={'text': 'one fun activity for parents during the holidays is to suggest an old ..., value=None),
 Task(output={'text': "krippendorf's tribe is a formula comedy . \ndone poorly , formulaic co..., value=None),
 Task(output={'text': "note : some may consider portions of the following text to be spoilers..., value=None),
 Task(output={'text': ' " if there\'s a beast in men , it meets its match in women , too . " ..., value=None),
 Task(output={'text': 'bruce willis and sixth sense director m . night shyamalan re-team to t..., value=None),
 